In [ ]:
%pip install pandas
%pip install numpy
%pip install sklearn
%pip install optuna
%pip install matplotlib

In [ ]:
import pandas as pd
import numpy as np
import sklearn
import tensorflow as tf
import math

from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers, datasets
from matplotlib import pyplot as plt

In [ ]:
import optuna

from optuna.integration import TFKerasPruningCallback
from optuna.trial import TrialState
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice

In [ ]:
(train_images, train_labels), (val_images, val_labels) = datasets.cifar10.load_data()
print(train_images.shape)
print(val_images.shape)

In [ ]:
train_images, test_images, train_labels, test_labels = train_test_split(
    train_images, train_labels, test_size=0.2, random_state=1337)

print(train_images.shape)
print(train_labels.shape)
print(val_images.shape)
print(val_labels.shape)
print(test_images.shape)
print(test_labels.shape)

In [ ]:
train_labels = keras.utils.to_categorical(train_labels)
val_labels = keras.utils.to_categorical(val_labels)
test_labels = keras.utils.to_categorical(test_labels)

train_images = train_images.astype('float32')
val_images = val_images.astype('float32')
test_images = test_images.astype('float32')
train_images = train_images / 255.0
val_images = val_images / 255.0
test_images = test_images / 255.0

In [ ]:
def create_model(trial):
    metrics = [
        "accuracy"
    ]
    
    dense_layers = trial.suggest_int("dense_layers", 1, 5)
    conv_layers = trial.suggest_int("conv_layers", 1, 5)
    dense_nodes = trial.suggest_categorical("dense_nodes", [512, 256, 128, 64, 32])
    conv_filters = trial.suggest_categorical("conv_filters", [64,32,16,8])
    
    model = keras.Sequential()
    model.add(layers.Input((32,32,3)))
    model.add(layers.Conv2D(conv_filters, (3,3), activation="relu", padding="same"))
    for i in range(1, conv_layers):
        model.add(layers.MaxPooling2D())
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.2))
        model.add(layers.Conv2D(conv_filters * (2**i), (3,3), activation="relu", padding="same"))
    model.add(layers.Flatten())
    model.add(layers.Dense(dense_nodes, activation="relu"))
    for i in range(1, dense_layers):
        model.add(layers.Dense(dense_nodes / 2**(math.ceil(i/2)), activation="relu"))
        if i % 2 == 0:
            model.add(layers.Dropout(0.2))
    model.add(layers.Dense(10, activation = "softmax"))

    loss = keras.losses.CategoricalCrossentropy()
    model.compile(loss = loss, optimizer = keras.optimizers.Adam(), metrics = metrics)
    return model

In [ ]:
def objective(trial):
    keras.backend.clear_session()
    model = create_model(trial)
    reduce_lr_callback = keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=5, min_lr=0.00001
    )
    callbacks = [
        TFKerasPruningCallback(trial, "val_loss"),
        reduce_lr_callback
    ]
    
    history = model.fit(
        train_images,
        train_labels,
        epochs=20,
        batch_size=64,
        validation_data=(val_images, val_labels),
        callbacks=callbacks
    )
    
    return history.history["val_loss"][-1]

In [ ]:
study = optuna.create_study(
    direction = "minimize", 
    study_name="test29",
    storage="sqlite:///example.db",
    pruner=optuna.pruners.MedianPruner(
        n_startup_trials=5, n_warmup_steps=10, interval_steps = 5
    ),
    sampler=optuna.samplers.TPESampler(
        n_startup_trials=5
    )
)
study.optimize(objective, n_trials=80)

In [ ]:
def show_result(study):

    pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))


In [ ]:
show_result(study)